In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from utils import calcu_lcoe
from shapely.geometry import Polygon
from Visual import MapViser
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

In [2]:
cell_attr = pd.read_csv('./data/vre/vre_attr.csv')[['lon','lat',
                                                    'cap_mid','t2m_min',
                                                    't2m_mean','t2m_max',
                                                    'road_dist','annual_cf',
                                                    'region',
                                                    'capex_tag']]

cell_attr['pv_cap'] = cell_attr['cap_mid']

In [4]:
geo = []

for i,r in cell_attr.iterrows():
    geo.append(Polygon([
        (r['lon'],r['lat']),
        (r['lon']+0.25,r['lat']),
        (r['lon']+0.25,r['lat']-0.25),
        (r['lon'],r['lat']-0.25)
    ]))
    
cell_attr = gpd.GeoDataFrame(cell_attr,geometry=geo,crs='EPSG:4326')

In [5]:
we_attr = pd.read_csv('./data/vre/desert_we.csv',index_col=['lon','lat'])
wss_config = pd.read_csv('./data/res/wss_config.csv',index_col=['lon','lat'])

In [7]:
temp_mean_upper = 26
temp_max_upper = 46

In [8]:
valid_cell = cell_attr.loc[(cell_attr['t2m_mean'] <= temp_mean_upper) & 
                           (cell_attr['t2m_max'] <= temp_max_upper)].copy(deep=True)

In [ ]:
valid_cell['pv_ratio'] = -1.0
valid_cell['we_ratio'] = -1.0
valid_cell['st_ratio'] = -1.0
valid_cell['equal_cap'] = -1.0

In [ ]:
for i,r in valid_cell.iterrows():
    lon = r['lon']
    lat = r['lat']
    
    pv_ratio = wss_config.at[(lon,lat),'pv_cap'] * 100
    we_ratio = wss_config.at[(lon,lat),'we_cap'] * 100
    st_ratio = wss_config.at[(lon,lat),'sto_cap'] * 100
    
    pv_cap = r['pv_cap']
    we_cap = we_attr.at[(lon,lat),'cap_mid']
    
    try:
        equal_cap = min(pv_cap/max(pv_ratio,0.01),we_cap/max(we_ratio,0.01))
        valid_cell.at[i,'pv_ratio'] = pv_ratio
        valid_cell.at[i,'we_ratio'] = we_ratio
        valid_cell.at[i,'st_ratio'] = st_ratio
        valid_cell.at[i,'equal_cap'] = equal_cap
    except:
        continue

In [ ]:
valid_cell = valid_cell.loc[valid_cell['equal_cap']>0]

valid_cell['gen'] = valid_cell['equal_cap'] * 8760

In [15]:
capex_config = {
    'low':{'China':{'pv':1500,'we':2500,'st':1000},
           'Africa':{'pv':5300*1500/3450,'we':8400*2500/4200,'st':8008*1000/2400},
            'North America':{'pv':7000*1500/3450,'we':11900*2500/4200,'st':8008*1000/2400},
            'South America':{'pv':5200*1500/3450,'we':5850*2500/4200,'st':9500*1000/2400},
            'Asia':{'pv':4081*1500/3450,'we':7800*2500/4200,'st':7000*1000/2400},
            'Oceania':{'pv':5155*1500/3450,'we':10655*2500/4200,'st':11130*1000/2400},
            'Europe':{'pv':7560*1500/3450,'we':10920*2500/4200,'st':8400*1000/2400},
            'India':{'pv':3400*1500/3450,'we':5882*2500/4200,'st':3700*1000/2400}},
    
    'mid':{'China':{'pv':2200,'we':3000,'st':1500},
           'Africa':{'pv':5300*2200/3450,'we':8400*3000/4200,'st':8008*1500/2400},
            'North America':{'pv':7000*2200/3450,'we':11900*3000/4200,'st':8008*1500/2400},
            'South America':{'pv':5200*2200/3450,'we':5850*3000/4200,'st':9500*1500/2400},
            'Asia':{'pv':4081*2200/3450,'we':7800*3000/4200,'st':7000*1500/2400},
            'Oceania':{'pv':5155*2200/3450,'we':10655*3000/4200,'st':11130*1500/2400},
            'Europe':{'pv':7560*2200/3450,'we':10920*3000/4200,'st':8400*1500/2400},
            'India':{'pv':3400*2200/3450,'we':5882*3000/4200,'st':3700*1500/2400}},
    
    
    'high':{'China':{'pv':2800,'we':3500,'st':2000},
            'Africa':{'pv':5300*2800/3450,'we':8400*3500/4200,'st':8008*2000/2400},
            'North America':{'pv':7000*2800/3450,'we':11900*3500/4200,'st':8008*2000/2400},
            'South America':{'pv':5200*2800/3450,'we':5850*3500/4200,'st':9500*2000/2400},
            'Asia':{'pv':4081*2800/3450,'we':7800*3500/4200,'st':7000*2000/2400},
            'Oceania':{'pv':5155*2800/3450,'we':10655*3500/4200,'st':11130*2000/2400},
            'Europe':{'pv':7560*2800/3450,'we':10920*3500/4200,'st':8400*2000/2400},
            'India':{'pv':3400*2800/3450,'we':5882*3500/4200,'st':3700*2000/2400}},
    
    '2024':{'China':{'pv':3450,'we':4200,'st':2400},
            'Africa':{'pv':5300,'we':8400,'st':8008},
            'North America':{'pv':7000,'we':11900,'st':8008},
            'South America':{'pv':5200,'we':5850,'st':9500},
            'Asia':{'pv':4081,'we':7800,'st':7000},
            'Oceania':{'pv':5155,'we':10655,'st':11130},
            'Europe':{'pv':7560,'we':10920,'st':8400},
            'India':{'pv':3400,'we':5882,'st':3700}
            },
    
    'freetrade':{'China':{'pv':2200,'we':3000,'st':1500},
              'Africa':{'pv':2200,'we':3000,'st':1500},
              'North America':{'pv':2200,'we':3000,'st':1500},
              'South America':{'pv':2200,'we':3000,'st':1500},
              'Asia':{'pv':2200,'we':3000,'st':1500},
              'Oceania':{'pv':2200,'we':3000,'st':1500},
              'Europe':{'pv':2200,'we':3000,'st':1500},
              'India':{'pv':2200,'we':3000,'st':1500}}
}

In [ ]:
for k in capex_config:
    valid_cell['lcoe_'+k] = -1.0
    
    for i,r in valid_cell.iterrows():
        capex_tag = r['capex_tag']
        
        opex = (capex_config[k][capex_tag]['pv'] * 0.005 + 
                capex_config[k][capex_tag]['we'] * 0.015 + 
                capex_config[k][capex_tag]['st'] * 0.005)
        
        capex = (capex_config[k][capex_tag]['pv'] * r['pv_ratio'] + 
                 capex_config[k][capex_tag]['we'] * r['we_ratio'] + 
                 capex_config[k][capex_tag]['st'] * r['st_ratio'])
        
        lcoe = calcu_lcoe(capex=capex,opex=opex,cf=1,years=25,disc_r=0.074)
        
        valid_cell.at[i,'lcoe_'+k] = lcoe
        

In [17]:
valid_cell.to_file('./data/res/valid_cell.gpkg')

In [18]:
valid_cell = valid_cell.loc[valid_cell['lcoe_mid']>0]
valid_cell = valid_cell.sort_values(by='lcoe_mid',ascending=True)
valid_cell['gen_cum'] = valid_cell['gen'].cumsum() * 1e-6

lcoe_low = valid_cell.loc[valid_cell['lcoe_low']>0].copy(deep=True)
lcoe_low = lcoe_low.sort_values(by='lcoe_low',ascending=True)
lcoe_low['gen_cum'] = lcoe_low['gen'].cumsum() * 1e-6

lcoe_high = valid_cell.loc[valid_cell['lcoe_high']>0].copy(deep=True)
lcoe_high = lcoe_high.sort_values(by='lcoe_high',ascending=True)
lcoe_high['gen_cum'] = lcoe_high['gen'].cumsum() * 1e-6

lcoe_2024 = valid_cell.loc[valid_cell['lcoe_2024']>0].copy(deep=True)
lcoe_2024 = lcoe_low.sort_values(by='lcoe_2024',ascending=True)
lcoe_2024['gen_cum'] = lcoe_2024['gen'].cumsum() * 1e-6

lcoe_ft = valid_cell.loc[valid_cell['lcoe_freetrade']>0].copy(deep=True)
lcoe_ft = lcoe_ft.sort_values(by='lcoe_freetrade',ascending=True)
lcoe_ft['gen_cum'] = lcoe_ft['gen'].cumsum() * 1e-6

In [ ]:
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = 25

colors = {
        'low':'#2e1a44',
        'mid':'#6a3d9a',
        'high':'#cab2d6',
        '2024':'#11ade6',
        'freetrade':'green'
}

fig,ax = plt.subplots(figsize=(14,18),dpi=400)

ax.plot(lcoe_low['lcoe_low']*1000/7, 
        lcoe_low['gen_cum'], 
        color=colors['low'],
        linewidth=3.5,
        label='CapEx (Low)')

ax.plot(valid_cell['lcoe_mid']*1000/7, 
        valid_cell['gen_cum'], 
        color=colors['mid'],
        linewidth=3.5,
        label='CapEx (Mid)')

ax.plot(lcoe_high['lcoe_high']*1000/7, 
        lcoe_high['gen_cum'], 
        color=colors['high'],
        linewidth=3.5,
        label='CapEx (High)')

ax.plot(lcoe_2024['lcoe_2024']*1000/7, 
        lcoe_2024['gen_cum'], 
        color=colors['2024'],
        linewidth=3.5,
        label='CapEx (2024)')

ax.set_xlim(20.0,160)
ax.set_ylim(-10,250)

ax.minorticks_on()

ax.tick_params(axis='y', 
               which='major', 
               length=8, 
               color='black')

ax.tick_params(axis='y', 
               which='minor', 
               length=4, 
               color='black')

ax.tick_params(axis='x', 
               which='minor', 
               length=4, 
               color='black')

ax.tick_params(axis='x', 
               which='major', 
               length=8, 
               color='black')

ax.legend(edgecolor='black',
          loc='lower right',
          frameon=False)


ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.set_ylabel('Cumulative generation (PWh/yr)')
ax.set_xlabel('LCOE ($/MWh)')
plt.savefig('./data/fig/Vis/lcoe_ele.png',bbox_inches='tight')
plt.show()

In [35]:
map_vis = MapViser()
world_land = gpd.read_file('./data/geo/Base/world_land.shp').to_crs('epsg:4326')
world = gpd.read_file('./data/geo/Base/world_map.shp').to_crs('epsg:4326')

In [36]:
desert = gpd.read_file('./data/geo/desert.shp').to_crs('epsg:4326')
desert['idx'] = range(desert.shape[0])

In [ ]:
valid_cell['lcoe_usd'] = valid_cell['lcoe_mid'] * 1000 / 7 # to us dollar

In [ ]:

map_vis.set_font()
map_vis.set_bound_color('gainsboro')

cmap = 'viridis'

fig,ax = plt.subplots(figsize=(12.6,8.4),dpi=600)

world.plot(ax=ax,facecolor='gainsboro',alpha=0.75,linewidth=0.45)

desert.plot(ax=ax,facecolor='silver',hatch='///')

valid_cell.plot(ax=ax,column='lcoe_usd',cmap=cmap,vmax=150,vmin=20)


ax.set_xlim(-125,148)
ax.set_ylim(-40,55)

map_vis.add_colorbar(fig=fig, 
                     loc=[0.45,0.28,0.2,0.0225],
                     vmax=150,
                     vmin=20,
                     cmap=cmap,
                     orientation='horizontal',
                     label='LCOE ($/MWh) of the WSS system in each grid cell')

ax.axis('off')

plt.savefig('./data/fig/Vis/wss_lcoe.png',bbox_inches='tight')

In [42]:
desert_cell = gpd.sjoin(left_df=desert,right_df=valid_cell,predicate='contains')

In [43]:
desert_equal_cap = pd.merge(desert, desert_cell.groupby(by='idx').sum(numeric_only=1).reset_index()[['idx','equal_cap']],on='idx')

In [ ]:

map_vis.set_font()
map_vis.set_bound_color('gainsboro')

cmap = 'rainbow'

fig,ax = plt.subplots(figsize=(12.6,8.4),dpi=600)

world.plot(ax=ax,facecolor='gainsboro',alpha=0.75,linewidth=0.45)

desert_equal_cap.plot(ax=ax,
                      column='equal_cap',
                      cmap=cmap,
                      vmax=500,
                      edgecolor='white',
                      lw=0.25)

ax.set_xlim(-125,148)
ax.set_ylim(-40,55)

map_vis.add_colorbar(fig=fig, 
                     loc=[0.45,0.28,0.2,0.0225],
                     vmax=500,
                     cmap=cmap,
                     orientation='horizontal',
                     label='Capacity potential (GW)')

ax.axis('off')

plt.savefig('./data/fig/Vis/wss_desert_equal_cap.png',bbox_inches='tight')